In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

/home/rodrigo/Documents/Rodrigo/Python_VE/tf_1_3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rodrigo/Documents/Rodrigo/Python_VE/tf_1_3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rodrigo/Documents/Rodrigo/Python_VE/tf_1_3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/hom

In [2]:
census = pd.read_csv('data/census_data.csv')

In [3]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
## Transform categorical data into numerical

cat_columns = census.select_dtypes(['object']).columns
census[cat_columns] = census[cat_columns].astype('category')
census[cat_columns] = census[cat_columns].apply(lambda x: x.cat.codes)

In [5]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,7,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,9,13,2,10,5,2,0,0,0,40,5,0


## Train and test split

In [6]:
# Train and test split
x_data = census.drop('income_bracket',axis=1)
labels = census['income_bracket']

x_train, x_test, y_train, y_test = train_test_split(x_data,labels,test_size = 0.3)

In [7]:
# Create fetuare columns
feat_cols = []
for column in x_data.columns:
    feat_cols.append(tf.feature_column.numeric_column(column))
    #if(str(census[column].dtype) == "int64"):   
    #else:
        #feat_cols.append(tf.feature_column.categorical_column_with_hash_bucket(column,hash_bucket_size=10))

In [8]:

input_func = tf.estimator.inputs.pandas_input_fn(x=x_train, y=y_train,batch_size=100,num_epochs=None,shuffle=True)
dnn_model = tf.estimator.DNNClassifier(hidden_units=[13,15,13],feature_columns=feat_cols,n_classes=2)
dnn_model.train(input_fn=input_func,steps=10000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpmcpyyfpp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fac10127ad0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.

INFO:tensorflow:loss = 34.735275, step = 5301 (0.181 sec)
INFO:tensorflow:global_step/sec: 543.251
INFO:tensorflow:loss = 51.442253, step = 5401 (0.185 sec)
INFO:tensorflow:global_step/sec: 494.393
INFO:tensorflow:loss = 31.203716, step = 5501 (0.201 sec)
INFO:tensorflow:global_step/sec: 579.153
INFO:tensorflow:loss = 35.39382, step = 5601 (0.173 sec)
INFO:tensorflow:global_step/sec: 516.188
INFO:tensorflow:loss = 35.401054, step = 5701 (0.193 sec)
INFO:tensorflow:global_step/sec: 536.47
INFO:tensorflow:loss = 34.960213, step = 5801 (0.187 sec)
INFO:tensorflow:global_step/sec: 420.345
INFO:tensorflow:loss = 38.165504, step = 5901 (0.238 sec)
INFO:tensorflow:global_step/sec: 495.845
INFO:tensorflow:loss = 34.580086, step = 6001 (0.201 sec)
INFO:tensorflow:global_step/sec: 503.872
INFO:tensorflow:loss = 35.507908, step = 6101 (0.199 sec)
INFO:tensorflow:global_step/sec: 513.718
INFO:tensorflow:loss = 39.48409, step = 6201 (0.195 sec)
INFO:tensorflow:global_step/sec: 555.566
INFO:tensorfl

In [9]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test, y=y_test,batch_size=100,num_epochs=1,shuffle=False)
dnn_model.evaluate(eval_input_func)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-11-29T19:03:42Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/tmpmcpyyfpp/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-11-29-19:03:42
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.83836627, accuracy_baseline = 0.7547344, auc = 0.88577807, auc_precision_recall = 0.7293798, average_loss = 0.35804945, global_step = 10000, label/mean = 0.24526563, loss = 35.691685, precision = 0.703335, prediction/mean = 0.25948054, recall = 0.5897329
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: /tmp/tmpmcpyyfpp/model.ckpt-10000


{'accuracy': 0.83836627,
 'accuracy_baseline': 0.7547344,
 'auc': 0.88577807,
 'auc_precision_recall': 0.7293798,
 'average_loss': 0.35804945,
 'label/mean': 0.24526563,
 'loss': 35.691685,
 'precision': 0.703335,
 'prediction/mean': 0.25948054,
 'recall': 0.5897329,
 'global_step': 10000}

In [10]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test,batch_size=len(x_test),num_epochs=1,shuffle=False)
pred_gen = dnn_model.predict(pred_input_func)
predictions = list(pred_gen)

final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpmcpyyfpp/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [11]:
 print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           0       0.87      0.92      0.90      7373
           1       0.70      0.59      0.64      2396

    accuracy                           0.84      9769
   macro avg       0.79      0.75      0.77      9769
weighted avg       0.83      0.84      0.83      9769

